# Tutorial Vision-视觉识别

以下内容适用于仿真

### 获取相机信息及图像

In [ ]:
import rospy
from arx5_control.control import RoboticArmAgent
# ROS节点初始化
NODE_NAME='arm_vision'
rospy.init_node(NODE_NAME)
arm = RoboticArmAgent(use_sim=True,node_name=NODE_NAME)
arm.go_to_named_or_joint_target('PickSim')


In [ ]:
from sensor_msgs.msg import Image,CameraInfo
# 获取相机参数，主要用到了分辨率作为参考
camrera_info:CameraInfo = rospy.wait_for_message("/camera/color/camera_info",
                                                   CameraInfo,timeout=2)
ImageSize = [camrera_info.width,camrera_info.height]
print(ImageSize)

In [ ]:
# 获取相机图像
frame_ros = rospy.wait_for_message("camera/color/image_raw",Image)

### 转换图像类型

In [ ]:
from cv_bridge import CvBridge
from arx5_vision.vision import StdVideo
frame  = CvBridge().imgmsg_to_cv2(frame_ros, "bgr8")  # Image图像转换为cv2格式
framecp = frame.copy()  # 为了避免原图被图像处理函数破坏，对原图进行复制，然后可以用复制后的图像进行处理
StdVideo.plt_imgs_show(frame)

In [ ]:
import cv2
# 将图像从BGR颜色空间转换为HSV颜色空间
img_hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
# 分离H、S、V三个通道
h, s, v = cv2.split(img_hsv)
# 展示图像
StdVideo.Show('out_put',[h,s,v],wait_time=0)

### 图像颜色分割

In [ ]:
# 执行如下代码可以通过滑动条调整视觉参数
HSV_L,HSV_H = StdVideo.color_thresh_determine(framecp)  # 阻塞，按esc退出

In [ ]:
_,binary_img = StdVideo.color_filter(framecp,HSV_L,HSV_H)
StdVideo.plt_imgs_show(binary_img)

### 图像轮廓提取

In [ ]:
contours = StdVideo.Contour.find(binary_img)
center_xy,bias_xyz,rotation_angle,box_int,cnt,w_h = StdVideo.Contour.NFC_F(contours,
                                        (4000,60000),3.0,(ImageSize[0]/2,529,20000))
if box_int is not None:
    StdVideo.Contour.draw(frame,[box_int])  # 在原图上绘制
    StdVideo.plt_imgs_show([frame,binary_img],dpi=100)

#### 退出ROS节点

In [ ]:
# 关闭视觉节点
rospy.signal_shutdown('end')